Ein einfaches RBAC-Beispiel mit einem Kubernetes-»User«
-------------------------------------------------------

![](https://d33wubrfki0l68.cloudfront.net/673dbafd771491a080c02c6de3fdd41b09623c90/50100/images/docs/admin/access-control-overview.svg)

Quelle: [https://kubernetes.io/docs/reference/access-authn-authz/controlling-access/](https://kubernetes.io/docs/reference/access-authn-authz/controlling-access/)

- - -

In dieser Übung legen wir einen normalen User an. Dazu erstellen wir ein X509-Client-Zertifikat welches den User authentifiziert.

Für weitere Informationen siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/rbac).

- - -

Zuerst müssen wir den Private Key für das User-Zertifikat generieren und diesen signieren.

Diese Aktionen erzeut die Dateien snoopy.pem` und snoopy.csr`.

In [ ]:
! rm -f snoopy* ~/.kube/config-snoopy
! openssl genrsa -out snoopy.pem 2048
! openssl req -new -key snoopy.pem -out snoopy.csr -subj "/CN=snoopy"
! ls -l snoopy*

- - -
Der CSR (Certificate Signing Request) ist in Kubernetes als Ressource zu erstellen.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: certificates.k8s.io/v1beta1
kind: CertificateSigningRequest
metadata:
  name: user-request-snoopy
spec:
  groups:
  - system:authenticated
  request: $(cat snoopy.csr | base64 | tr -d '\n')
  usages:
  - digital signature
  - key encipherment
  - client auth
%EOF%

Zu beglaubigen und zu signieren:

In [ ]:
! kubectl certificate approve user-request-snoopy
! kubectl get csr user-request-snoopy -o jsonpath='{.status.certificate}' | base64 -d > snoopy.crt  

Der erste Teil ist erledigt, wir haben nun drei Dateien und ein beglaubigtes Zertifikat in Kubernetes:

In [ ]:
! ls -l snoopy*
! kubectl get csr user-request-snoopy

***
**Konfiguration**

Damit wir den User Account verwenden können, benötigen wir eine entsprechende K8s-Konfiguration, diese steht nachher in `.kube/config-snoopy`:

In [ ]:
! kubectl --kubeconfig ~/.kube/config-snoopy config set-cluster kubernetes --insecure-skip-tls-verify=true --server=$(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }')
! kubectl --kubeconfig ~/.kube/config-snoopy config set-credentials snoopy --client-certificate=snoopy.crt --client-key=snoopy.pem --embed-certs=true

Der erste Befehl legt die Datei `.kube/config-snoopy` mit den Cluster Informationen an, der zweite Befehl ergänzt die User Informationen.

Anschliessend Erzeugen wir einen Context `snoopy` und wechseln auf diesen.

In [ ]:
! kubectl --kubeconfig ~/.kube/config-snoopy config set-context snoopy --cluster=kubernetes --user=snoopy 
! kubectl --kubeconfig ~/.kube/config-snoopy config use-context snoopy

Da noch die entsprechende Rolle und das Binding fehlt sollte folgender Befehl eine Fehler ausgeben:

In [ ]:
! kubectl --kubeconfig ~/.kube/config-snoopy get pods 

- - -
### Role und RoleBinding erzeugen

Damit der neue User Zugriff auf die Pods erhält, ist die nachfolgende YAML Datei zu erstellen

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
kind: Role
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  namespace: default
  name: pod-reader-role
rules:
- apiGroups: [""]
  resources: ["pods"]
  verbs: ["get", "watch", "list"]
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  name: pod-reader-rolebinding
  namespace: default
subjects:
- kind: User
  name: snoopy
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: pod-reader-role
  apiGroup: rbac.authorization.k8s.io
%EOF%


Anschliessend muss die Abfrage von `Pods` in der Namespace `default` mit dem User `snoopy` funktionieren:

In [ ]:
! kubectl --kubeconfig ~/.kube/config-snoopy get pods

- - -
Aufräumen

In [ ]:
! rm -f snoopy* ~/.kube/config-snoopy
! kubectl delete Role/pod-reader-role
! kubectl delete RoleBinding/pod-reader-rolebinding